In [18]:
!pip install pandas pyarrow

In [15]:
import pandas as pd
import os

Data cleaning and processing. All comments can be found on explanation.ipynb

In [16]:
directory = "grouped"

if not os.path.exists(directory):
    os.makedirs(directory)


def calculate_week_number(date):
    first_day_of_year = pd.Timestamp(year=date.year, month=1, day=1)
    first_day_weekday = first_day_of_year.weekday() 

    if first_day_weekday in [0, 1, 2]:  
        return date.strftime('%W')
    else:  
        week_num = int(date.strftime('%W')) - 1
        return max(week_num, 0)  
    
for i in range(1, 13):
    if i < 10:
        i = f'0{i}'
    file_s = f'generative\sales_{i}_2023.csv'
    sales_df = pd.read_csv(file_s, delimiter=',', engine='pyarrow', dtype_backend='pyarrow', dtype={'CARD': 'string[pyarrow]'})

    sales_df['DATE'] = pd.to_datetime(sales_df['DATE'], format='%d.%m.%Y')
    sales_df['QUANTITY'] = sales_df['QUANTITY'].astype(str).str.replace(',', '.').astype('float32[pyarrow]')
    sales_df['AMOUNT'] = sales_df['AMOUNT'].astype(str).str.replace(',', '.').astype('float32[pyarrow]')
    sales_df['GROSSAMOUNT'] = sales_df['GROSSAMOUNT'].astype(str).str.replace(',', '.').astype('float32[pyarrow]')
    sales_df['BONUSES_SPENT'] = sales_df['GROSSAMOUNT'] - sales_df['AMOUNT']

    sales_df = sales_df.groupby(['DATE', 'PRODUCT_ID', 'BONUSES_SPENT', 'GROSSAMOUNT', 'AMOUNT']).agg({'QUANTITY': 'sum'}).reset_index()
    sales_df = sales_df.rename(columns={'GROSSAMOUNT': 'COST (without bonuses)'})

    prodacts_df = pd.read_csv('generative\Products.csv', delimiter=',', engine='pyarrow', dtype_backend='pyarrow')

    merged_df = sales_df.set_index('PRODUCT_ID').join(prodacts_df.set_index('PRODUCT_ID'))
    merged_df = merged_df.reset_index()
    merged_df['Week day'] = merged_df['DATE'].dt.strftime('%a')
    merged_df['Week number'] = merged_df['DATE'].apply(calculate_week_number).astype(int)
    merged_df['Month number'] = merged_df['DATE'].dt.month

    desired_order = ['Week number','Month number', 'Week day', 'BONUSES_SPENT','COST (without bonuses)','AMOUNT','QUANTITY','PRODUCT','CATEGORY','SUBCATEGORY']
    merged_df = merged_df[desired_order]

    subcategory_grouped_df = merged_df.groupby(['CATEGORY', 'SUBCATEGORY', 'Week number', 'Month number', 'Week day']).agg({
        'QUANTITY': 'sum',
        'BONUSES_SPENT': 'sum',
        'COST (without bonuses)': 'sum'
    }).reset_index()

    subcategory_grouped_df["BONUSES_RECIVE"] = subcategory_grouped_df["COST (without bonuses)"] * 0.025
    subcategory_grouped_df["BONUSES_RECIVE"] = subcategory_grouped_df["BONUSES_RECIVE"].apply(lambda x: '{:.2f}'.format(x))
    subcategory_grouped_df.to_csv(f'grouped/grouped_by_subcat_{i}_2023.csv', index=False)


Merging data for dashboard

In [19]:
df = pd.read_csv('grouped\grouped_by_subcat_01_2023.csv', delimiter=',', engine='pyarrow', dtype_backend='pyarrow',  dtype={'CARD': 'string[pyarrow]'})
for i in range(2,13):
    if i < 10:
        i = f'0{i}'
    file_s = f'grouped\grouped_by_subcat_{i}_2023.csv'
    df = pd.concat([df, pd.read_csv(file_s, delimiter=',', engine='pyarrow', dtype_backend='pyarrow',  dtype={'CARD': 'string[pyarrow]'})], ignore_index=True)
df_sorted = df.sort_values(by=['CATEGORY', 'SUBCATEGORY', 'Week number'])
df_sorted.to_csv(f'grouped/grouped_by_subcat_2023.csv', index=False)